# Classificação de Qualidade de Vinhos

- Possuimos 2 datasets com vinhos tintos e brancos do norte de Portugal. O objetivo é fazer uma previsão da qualidade do vinho baseado nos testes químicos feitos
- Usaremos o algoritmo de Naive Bayes para classificar os vinhos.